In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import networkx as nx
import pyomo.environ as pe
import pyomo.opt as po

## Define the Graph

In [ ]:
nodes = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
edges = {(0, 1), (0, 2), (0, 3), (1, 4),
         (1, 6), (2, 1), (2, 3), (2, 5),
         (3, 5), (4, 2), (5, 7), (5, 8),
         (6, 4), (6, 7), (6, 9), (7, 4),
         (7, 9), (8, 3), (8, 7), (8, 9)}
distance = {(0, 1): 40, (0, 2):  8, (0, 3): 10, (1, 4):  6,
            (1, 6): 10, (2, 1):  4, (2, 3): 12, (2, 5):  2,
            (3, 5):  1, (4, 2):  2, (5, 7):  4, (5, 8):  3,
            (6, 4):  8, (6, 7): 20, (6, 9):  1, (7, 4):  0,
            (7, 9): 20, (8, 3):  6, (8, 7): 10, (8, 9):  2}

## Visualize Graph with NetworkX

In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(list(nodes))
graph.add_edges_from(list(edges))
pos = ({0: (4, 0), 1: (0, 0), 2: (4, 2), 3: (8, 0), 4: (2, 4),
        5: (6, 4), 6: (0, 8), 7: (4, 6), 8: (8, 8), 9: (4, 8)})

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color='lightgray',
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distance,
                             font_size=16, rotate=False)
plt.axis('off')
plt.show()

## Define Sets of Incoming and Outgoing Edges
This data preprocessing step makes our lives easier when we define the flow balance constraints. For large graphs, doing this ahead of time in preprocessing (as opposed to on-the-fly during model construction) reduces instance build time from $\mathcal{O}(n^2)$ to $\mathcal{O}(n)$.

In [ ]:
Vm = defaultdict(set)
Vp = defaultdict(set)
for (i, j) in edges:
    Vm[i].add(j)
    Vp[j].add(i)

## Formulate the Shortest Path LP

Let $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ be the graph. Let $s$ and $t$ be the start and terminal nodes, respectively, and let $d_{ij}$ denote distance from node $i$ to node $j$. As variables, let $x_{ij}$ be a 0-1 variable indicating the decision to permit flow across edge $(i, j)$. We also introduce the following notation to describe adjacent nodes on incoming and outgoing edges.

\begin{aligned}
    \mathcal{V}^-(i) = \{j\ |\ (i, j) \in \mathcal{E}\} \\
    \mathcal{V}^+(i) = \{j\ |\ (j, i) \in \mathcal{E}\}
\end{aligned}

We require one unit of flow to enter the graph at start node $s$ and one unit of flow to leave the graph at terminal node $t$. At every other node, we require flow balance. Our objective is to minimize the length of the path satisfying the constraints. That is, we find the distance-weighted sum of the $x_{i,j}$ variables, the shortest path.

$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{(i, j) \in \mathcal{E}} d_{ij} x_{ij} && \\
\text{subject to  }
& \sum_{j \in \mathcal{V}^-(s)} x_{sj} = 1 && \\
& \sum_{j \in \mathcal{V}^+(t)} x_{jt} = 1 && \\
& \sum_{j \in \mathcal{V}^+(i)} x_{ji} = \sum_{j \in \mathcal{V}^-(i)} x_{ij},
&& \qquad \forall i \in \mathcal{V} \setminus \{s, t\} \\
& x_{ij} \in \{0,1\}, 
&& \qquad \forall (i, j) \in \mathcal{E}
\end{alignat*}
$$

## Create the Model Object

In [ ]:
model = pe.ConcreteModel()

## Define the Sets
Notice use of `within` when defining the set of edges. This ensures that $\mathcal{E} \subseteq \mathcal{V} \times \mathcal{V}$. While this is not necessary, it is good practice and will help identify data errors.

In [ ]:
model.nodes = pe.Set(initialize=nodes)
model.edges = pe.Set(within=model.nodes*model.nodes, initialize=edges)

## Define the Parameters
The first argument of `pe.Param` is the set in which the parameters are indexed. For example, distances are defined on the set of edges. The `initialize` argument here should be a dictionary-like construct. The dictionary keys should match align with the index set and the values are... well... the values. Notice also that $\mathcal{V}^+(\cdot)$ and $\mathcal{V}^-(\cdot)$ are indexed *sets*, not technically *parameters*. Pyomo does not directly support "jagged" sets like this, but using `pe.Param` is one way to implement such a feature.

In [ ]:
model.Vm = pe.Param(model.nodes, initialize=Vm, default=set(), within=pe.Any)
model.Vp = pe.Param(model.nodes, initialize=Vp, default=set(), within=pe.Any)
model.s = 0
model.t = 9
model.distance = pe.Param(model.edges, initialize=distance)

# Define the Variables
Just as it was with `pe.Param`, the first argument to `pe.Var` is the index set in which the variables are indexed. Optionally, we may define the domain of the variable set. Here, we require each $x_{i,j}$ to be binary -- either 0 or 1. If not explicitly defined, the domain is assumed to be the set of all real numbers (`pe.Reals`). The constraint matrix is totally unimodular. Consequently, the variables will assume integer values even if they are relaxed to the $[0, 1]$ interval. Below, we implement the relaxation using a combination of `domain` and `bounds`.

In [ ]:
model.x = pe.Var(model.edges, domain=pe.Reals, bounds=(0, 1))

## Define the Objective
Unlike the previous examples in which we defined the objective using an `expr`, we now use a `rule`. What's the difference? The former just requires expression. The latter requires a function that returns an expression. And why would make it more complex by involving a function? Because it's modular! (Suppose we wanted to use this objective function in a bunch of other models. We can define it once in the function, then add it to a model with one line!)

In [ ]:
def shortest_path(model):
    return sum(model.distance[i, j] * model.x[i, j]
               for (i, j) in model.edges)

model.obj_shortest_path = pe.Objective(sense=pe.minimize, rule=shortest_path)

## Define the Constraints
Similarly, we define the constraints using a rule (i.e., a function) instead of a bunch of expressions. We do this with a `pe.Constraint` instead of a `pe.ConstraintList` object. The positional arguments passed to `pe.Constraint` are the index sets over which the constraints are defined. In our case, there is one constraint per node, so we pass the just set of nodes.

In the general case, the constraints may be indexed in multiple node sets, say $A_1$ through $A_m$. Suppose we write the following function to use as our rule:

```
def constraint_rule(model, i_1, i_2, ..., i_m):
    ...
    return some_condition_based_on_arguments

```

Then

```
model.con = pe.Constraint(model.A_1, model.A_2, ..., model.A_m,
                          rule=constraint_rule)
```



is basically the same as

```
model.con = ConstraintList()
for i_1 in model.A_1:
    for i_2 in model.A_2:
        ...
            for i_m in model.A_m:
                model.con.add(constraint_rule(model, i_1, i_2, ... i_m))
```

That is, both impose a constraint for each combination of indices in the Cartesian product of all index sets.

This functional programming approach has a few advantages:
1. The nested for-loops we would otherwise have to write are handled implicitly.
2. We are able to access each constraint later by the defining combination of indices.
3. Again, modularity!

In [ ]:
def flow_balance(model, i):
    flow_in = sum([model.x[j, i] for j in model.Vp[i]])
    flow_out = sum([model.x[i, j] for j in model.Vm[i]])
    if i == model.s:
        constraint = (flow_out == 1)
    elif i == model.t:
        constraint = (flow_in == 1)
    else:
        constraint = (flow_in == flow_out)
    return constraint

model.con_flow_balance = pe.Constraint(model.nodes, rule=flow_balance)

## Solve and Postprocess

In [ ]:
solver = po.SolverFactory('cbc')
result = solver.solve(model, tee=True)

Interpret the output to determine the shortest path.

In [ ]:
i = int(model.s)
path_nodes = [i]
path_edges = []
stop = False
while not stop:
    for j in model.Vm[i]:
        if model.x[i, j].value == 1:
            if j == int(model.t):
                stop = True
            path_nodes.append(j)
            path_edges.append((i, j))
            i = j
            break

display(path_nodes)

## Visualize Solution with NetworkX

In [ ]:
node_colors = ['lightblue' if i in path_nodes else 'lightgray'
               for i in graph.nodes()]
edge_colors = ['blue' if (i, j) in path_edges else 'black'
               for (i, j) in graph.edges()]
edge_widths = [2 if (i, j) in path_edges else 1
               for (i, j) in graph.edges()]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25,
                       edge_color=edge_colors, width=edge_widths)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distance,
                             font_size=16, rotate=False)
plt.axis('off')
plt.show()